In [1]:
import os
import re
import json
import warnings
import pandas as pd
pd.options.mode.chained_assignment = None
from sec_api import QueryApi, XbrlApi

In [ ]:
api_key = "e3d46df2271d81a4b7a71153aef74fa5e7dae956314122b155767267425b36a6"

# Query 20 most recent 10Q Info

In [ ]:
# queryAPI = QueryApi(api_key=api_key)

In [ ]:
# query = {"query": {
#             "query_string": {
#                 "query": "ticker:TSLA AND formType:\"10-Q\"",
#                     }
#                 },
#          "from": "0",
#          "size": 4*5,
#          "sort": [{"filedAt": {"order": "desc"}}]
#             }

In [ ]:
### NOTE: USES UP MONTHLY API CALLS LIMIT
# response = queryAPI.get_filings(query)
# response_parsed = pd.DataFrame.from_records(response["filings"])

In [ ]:
# response_parsed.to_csv("TSLA_10Q_INFO_2017Q1_2023Q2.csv", index=False)
response_parsed = pd.read_csv("TSLA_10Q_INFO_2017Q1_2023Q2.csv")

# Parse 10Q

In [ ]:
# xbrlAPI = XbrlApi(api_key)

In [ ]:
# %%time
### NOTE: USES UP MONTHLY API CALLS LIMIT
# for date, url in response_parsed[["periodOfReport", "linkToFilingDetails"]].values:
#     xbrl_json = xbrlAPI.xbrl_to_json(url)
    
#     with open(f"tesla_10Q_{date}.json", "w") as f:
#         json.dump(xbrl_json, f)

## Extract Data (Dev)

In [ ]:
with open("tesla_10Q_2023-03-31.json", "r") as f:
    xbrl_json = json.load(f)

In [ ]:
current_date = xbrl_json["CoverPage"]["DocumentPeriodEndDate"]
current_date

In [ ]:
BalanceSheets = xbrl_json["BalanceSheets"]
StatementsOfIncome = xbrl_json["StatementsOfIncome"]
StatementsOfCashFlows = xbrl_json["StatementsOfCashFlows"]
# StatementsOfShareholdersEquity = xbrl_json["StatementsOfShareholdersEquity"]

## Balance Sheets

In [2]:
def balancesheets_to_df(BalanceSheets):
    """
    """
    df_list = []

    for name in BalanceSheets.keys():
        try:
            df = pd.DataFrame.from_records(BalanceSheets[name])
            df["period"] = df["period"].apply(lambda x: x["instant"] if isinstance(x, dict) else x)
            df["name"] = name
            df_list.append(df)
        except Exception as ex:
            print(name, "---", ex)
    
    return pd.concat(df_list, axis=0)

In [ ]:
balancesheet_df = balancesheets_to_df(BalanceSheets)
set(BalanceSheets.keys()).difference(set(balancesheet_df.name))

In [ ]:
(balancesheet_df.query("segment.isna()").value_counts(["name", "period"]) == 1).mean()

In [ ]:
# balancesheet_df.query("segment.isna() and period == @current_date")

## Statements of Income

In [3]:
def incomestatement_to_df(StatementsOfIncome):
    """
    """
    df_list = []

    for name in StatementsOfIncome.keys():
        try:
            df = pd.DataFrame.from_records(StatementsOfIncome[name])
            df["startDate"] = df["period"].apply(lambda x: x["startDate"] if isinstance(x, dict) else x)
            df["endDate"] = df["period"].apply(lambda x: x["endDate"] if isinstance(x, dict) else x)
            df["name"] = name
            df_list.append(df)
        except Exception as ex:
            print(name, "---", ex)
    
    return pd.concat(df_list, axis=0)

In [ ]:
income_df = incomestatement_to_df(StatementsOfIncome)
set(StatementsOfIncome.keys()).difference(set(income_df.name))

In [ ]:
(income_df.query("segment.isna()").value_counts(["name", "startDate", "endDate"]) == 1).mean()

In [ ]:
(income_df.query("segment.isna()").value_counts(["endDate", "name"]) == 2).mean()

In [ ]:
# (income_df.query("segment.isna() and endDate == @current_date")
#  .sort_values(["name", "startDate"]).groupby(["name"]).tail(1))

## Extract Data (prod.)

In [5]:
%%time
files = [i for i in os.listdir() if re.match(".*.json$", i)]
bs_lst, inc_lst = [], []

for file in files:
    print(file)
    with open(file, "r") as f:
        xbrl_json = json.load(f)
    
    current_date = xbrl_json["CoverPage"]["DocumentPeriodEndDate"]
    BalanceSheets = xbrl_json["BalanceSheets"]
    StatementsOfIncome = xbrl_json["StatementsOfIncome"]
    
    balancesheet_df = balancesheets_to_df(BalanceSheets)
    balancesheet_df = balancesheet_df.query("segment.isna() and period == @current_date")
    bs_lst.append(balancesheet_df)
    
    income_df = incomestatement_to_df(StatementsOfIncome)
    income_df = income_df.query("endDate == @current_date")
#     income_df = (income_df
#                  .query("segment.isna() and endDate == @current_date")
#                  .sort_values(["name", "startDate"])
#                  .groupby(["name"])
#                  .tail(1))
    inc_lst.append(income_df)

tesla_10Q_2017-03-31.json
tesla_10Q_2017-06-30.json
tesla_10Q_2017-09-30.json
tesla_10Q_2018-03-31.json
tesla_10Q_2018-06-30.json
tesla_10Q_2018-09-30.json
tesla_10Q_2019-03-31.json
tesla_10Q_2019-06-30.json
tesla_10Q_2019-09-30.json
tesla_10Q_2020-03-31.json
tesla_10Q_2020-06-30.json
tesla_10Q_2020-09-30.json
tesla_10Q_2021-03-31.json
tesla_10Q_2021-06-30.json
tesla_10Q_2021-09-30.json
tesla_10Q_2022-03-31.json
tesla_10Q_2022-06-30.json
tesla_10Q_2022-09-30.json
tesla_10Q_2023-03-31.json
tesla_10Q_2023-06-30.json
CPU times: total: 3.58 s
Wall time: 3.61 s


In [30]:
income_df = pd.concat(inc_lst, axis=0)
days_diff = (pd.to_datetime(income_df["endDate"]) - pd.to_datetime(income_df["startDate"])).dt.days
income_df = income_df[(days_diff <= 95) & (days_diff >= 87)]

In [35]:
(income_df
 .pivot(index=["startDate", "endDate"], columns=["name"], values="value")
 .isna().mean(axis=0)
 .sort_values())

ValueError: Index contains duplicate entries, cannot reshape

In [34]:
balancesheet_df = pd.concat(bs_lst, axis=0)

(balancesheet_df
 .drop_duplicates(["period", "name"])
 .pivot(index=["period"], columns=["name"], values="value")
 .isna().mean(axis=0)
 .sort_values())

name
AccountsPayableCurrent                                   0.00
RetainedEarningsAccumulatedDeficit                       0.00
RedeemableNoncontrollingInterestEquityCarryingAmount     0.00
PropertyPlantAndEquipmentNet                             0.00
PrepaidExpenseAndOtherAssetsCurrent                      0.00
OtherLiabilitiesNoncurrent                               0.00
OtherAssetsNoncurrent                                    0.00
MinorityInterest                                         0.00
LiabilitiesCurrent                                       0.00
StockholdersEquity                                       0.00
Liabilities                                              0.00
InventoryNet                                             0.00
IntangibleAssetsNetExcludingGoodwill                     0.00
Goodwill                                                 0.00
LiabilitiesAndStockholdersEquity                         0.00
AssetsCurrent                                            0.00
Acc